<a href="https://colab.research.google.com/github/hajsf/Time-Series-Forecast/blob/main/04_DefineFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install the Hijri converter package (if not installed)

In [ ]:
!pip install hijri-converter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import glob
import numpy as np
from hijri_converter import Hijri, Gregorian
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

**Read the data files**


*   Define correct seperator, tab in our example
*   Use dtype to enforce data type mainy in casefield field has mix data types
*   Use low_memory=False so you do not mind whatever type the undefined fields are read in they have mix data





In [ ]:
pathData = "/content/drive/MyDrive/ColabNotebooks/dataset/02_ProcessedData/"
pathExternal = "/content/drive/MyDrive/ColabNotebooks/dataset/03_ExternalData/"


In [ ]:
with open(pathData + 'data.pk', 'rb') as f:
  df = pickle.load(f)
print("number of records:", len(df))

number of records: 7288431


In [ ]:
df

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,TTL_VAL,TTL_CST
0,0020,13,2022-01-12,1.0,35.0,26.25
1,0020,13,2022-02-08,1.0,35.0,26.25
2,0020,13,2022-02-27,2.0,70.0,52.50
3,0020,13,2022-04-07,1.0,35.0,26.25
4,0020,13,2022-04-16,1.0,35.0,26.25
...,...,...,...,...,...,...
1252033,z99,18,2018-12-26,3.0,60.0,51.00
1252034,z99,18,2018-12-27,2.0,40.0,34.00
1252035,z99,18,2018-12-30,5.0,100.0,85.00
1252036,z99,18,2018-12-31,3.0,60.0,51.00


Test some data to ensure everything is read correctly

**Feature Creation**

In [ ]:
def hijriDate(row):  
    y = row['DATE1'].year
    m = row['DATE1'].month
    d = row['DATE1'].day
    return Gregorian(y, m, d).to_hijri()

In [ ]:
def isRamadan(dt):  
    if dt.month == 9:
      return 1
    else:
      return 0

In [ ]:
def isRamadanEnd(dt):  
    if dt.month == 9 and dt.day >= 21:
      return 1
    else:
      return 0

In [ ]:
def isEid(dt):  
    if (dt.month == 10 and dt.day < 4) or (dt.month == 12 and dt.day < 9 and dt.day > 14):
      return 1
    else:
      return 0

In [ ]:
def isWeekend(dt):  
    if dt.weekday() == 4 or dt.weekday() == 5: # Monday is 0
      return 1
    else:
      return 0

In [ ]:
def isSalaryweek(dt):  
    if dt.day >= 25:
      return 1
    else:
      return 0

In [ ]:
def isNationalday(dt):  
    if (dt.month == 2 and dt.day == 22) or (dt.month == 9 and dt.day == 23) :
      return 1
    else:
      return 0

Convert DATE1 from string to date format

In [ ]:
df['DATE1'] = pd.to_datetime(df['DATE1'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
#df = df.assign(isSalaryweek=lambda df: 1 if df['DATE1'].day >= 25 else 0)
# or usethe cusom function above
df['isSalaryweek'] = df['DATE1'].apply(lambda dt: isSalaryweek(dt))

In [ ]:
df.head()

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,TTL_VAL,TTL_CST,isSalaryweek,isWeekend,isNationalday
0,0020,13,2022-01-12,1.0,35.0,26.25,0,0,0
1,0020,13,2022-02-08,1.0,35.0,26.25,0,0,0
2,0020,13,2022-02-27,2.0,70.0,52.50,1,0,0
3,0020,13,2022-04-07,1.0,35.0,26.25,0,0,0
4,0020,13,2022-04-16,1.0,35.0,26.25,0,1,0


In [ ]:
df['isWeekend'] = df['DATE1'].apply(lambda dt: isWeekend(dt))

In [ ]:
df['isNationalday'] = df['DATE1'].apply(lambda dt: isNationalday(dt))

In [ ]:
# get the hijri equivalent date, day and month
df['hijri'] = df.apply(lambda row: hijriDate(row), axis=1)

In [ ]:
df['isRamadan'] = df['hijri'].apply(lambda dt: isRamadan(dt))

In [ ]:
df['isRamadanEnd'] = df['hijri'].apply(lambda dt: isRamadanEnd(dt))

In [ ]:
df['isEid'] = df['hijri'].apply(lambda dt: isEid(dt))

In [ ]:
df.head()

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,TTL_VAL,TTL_CST,isSalaryweek,isWeekend,isNationalday,hijri,isRamadan,isRamadanEnd,isEid
0,0020,13,2022-01-12,1.0,35.0,26.25,0,0,0,1443-06-09,0,0,0
1,0020,13,2022-02-08,1.0,35.0,26.25,0,0,0,1443-07-07,0,0,0
2,0020,13,2022-02-27,2.0,70.0,52.50,1,0,0,1443-07-26,0,0,0
3,0020,13,2022-04-07,1.0,35.0,26.25,0,0,0,1443-09-06,1,0,0
4,0020,13,2022-04-16,1.0,35.0,26.25,0,1,0,1443-09-15,1,0,0


In [ ]:
df['Monthday'] = df['DATE1'].apply(lambda dt: dt.day)
# or
# df['Monthday'] = df.index.day

In [ ]:
df.head()

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,TTL_VAL,TTL_CST,isSalaryweek,isWeekend,isNationalday,hijri,isRamadan,isRamadanEnd,isEid,Monthday
0,0020,13,2022-01-12,1.0,35.0,26.25,0,0,0,1443-06-09,0,0,0,12
1,0020,13,2022-02-08,1.0,35.0,26.25,0,0,0,1443-07-07,0,0,0,8
2,0020,13,2022-02-27,2.0,70.0,52.50,1,0,0,1443-07-26,0,0,0,27
3,0020,13,2022-04-07,1.0,35.0,26.25,0,0,0,1443-09-06,1,0,0,7
4,0020,13,2022-04-16,1.0,35.0,26.25,0,1,0,1443-09-15,1,0,0,16


In [ ]:
df['hour'] = df.index.hour
df['Weekday'] = df.index.dayofweek
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['year'] = df.index.year
df['dayofyear'] = df.index.dayofyear
df['weekofyear'] = df.index.isocalendar().week

AttributeError: ignored

Import school days and calendar related activities that could impact all items and sales centers and add them to the features dataframe

In [ ]:
calenderFeatures = pd.read_csv(pathExternal +'calenderFeatures.csv', sep=',', lineterminator='\r', low_memory=False) # , dtype={"DATE1": 'string', "SLS_CNTR_ID": "string", "ITEM_ID": "string"}

In [ ]:
calenderFeatures = calenderFeatures.replace('\n','', regex=True)

In [ ]:
calenderFeatures.head()

,DATE1,Holiday,AcademicCalendarSemester
0,2016-01-01,0,1
1,2016-01-02,0,1
2,2016-01-03,0,1
3,2016-01-04,0,1
4,2016-01-05,0,1


In [ ]:
calenderFeatures['DATE1'] = pd.to_datetime(calenderFeatures['DATE1'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df = pd.merge(df, 
              calenderFeatures, 
              on =['DATE1'], 
              how ='inner')

In [ ]:
df.head()

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,TTL_VAL,TTL_CST,isSalaryweek,isWeekend,isNationalday,hijri,isRamadan,isRamadanEnd,isEid,Monthday,Holiday,AcademicCalendarSemester
0,0020,13,2022-01-12,1.0,35.000000,26.25,0,0,0,1443-06-09,0,0,0,12,0,0
1,0035,13,2022-01-12,3.0,105.000000,78.75,0,0,0,1443-06-09,0,0,0,12,0,0
2,0035,28,2022-01-12,2.0,70.000000,52.50,0,0,0,1443-06-09,0,0,0,12,0,0
3,011810029,18,2022-01-12,1.0,30.000000,25.50,0,0,0,1443-06-09,0,0,0,12,0,0
4,100000103,48,2022-01-12,1.0,115.652174,68.00,0,0,0,1443-06-09,0,0,0,12,0,0


In [ ]:
df = df.drop(columns=['hijri', 'TTL_VAL', 'TTL_CST'])

In [ ]:
df.head()

,ITEM_ID,SLS_CNTR_ID,DATE1,QTY,isSalaryweek,isWeekend,isNationalday,isRamadan,isRamadanEnd,isEid,Monthday,Holiday,AcademicCalendarSemester
0,0020,13,2022-01-12,1.0,0,0,0,0,0,0,12,0,0
1,0035,13,2022-01-12,3.0,0,0,0,0,0,0,12,0,0
2,0035,28,2022-01-12,2.0,0,0,0,0,0,0,12,0,0
3,011810029,18,2022-01-12,1.0,0,0,0,0,0,0,12,0,0
4,100000103,48,2022-01-12,1.0,0,0,0,0,0,0,12,0,0


In [ ]:
with open(pathData + 'readyForTraining.pk', 'wb') as f:
  pickle.dump(df, f)

**Below to be implemented in the future**

Import selective days and calendar related activities that could impact selective items and sales centers and add them to the features dataframe

In [ ]:
selectiveFeatures = pd.read_csv(pathExternal +'selectiveFeatures.csv', sep='\t', lineterminator='\r', low_memory=False) # , dtype={"DATE1": 'string', "SLS_CNTR_ID": "string", "ITEM_ID": "string"}

In [ ]:
df = pd.merge(df, 
              calenderFeatures, 
              on =['DATE1', 'ITEM_ID', 'SLS_CNTR_ID'], 
              how ='inner')

In [ ]:
# save feature csv data to another file
df.to_csv(path+"features.csv",index=False)

**Visualize our Feature / Target Relationship**

In [ ]:
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [20, 5] #  [width, height]  
sns.boxplot(x = df.index.get_level_values('DATE1'),  # ('SLS_CNTR_ID')
            y = 'QTY',
            data = df, palette="Blues", width=0.3)